# Relationship Between Magnitude And Multiplicity

In [78]:
import pandas as pd
import numpy as np
import datetime as dt

In [79]:
csv_file = "../datasets/All (1960-2023).csv"
usgs = pd.read_csv(csv_file, sep = ',', lineterminator='\n', dtype={'time':str})

C:\Users\Vishal\AppData\Local\Temp\ipykernel_3344\3011563604.py:2: DtypeWarning:

Columns (1,2,3,4,6,7,8,9,15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.



## Data Filtering
1. Converting the date columns to datetime
2. Date > 1960-01-01 and < 2023-01-01
3. Longitude > -123 and < -113
4. Latitude > 29 and < 39

In [80]:
#filter the dataset by Date > 1960-01-01 and Date < 2023-01-1 
usgs["time"] = pd.to_datetime(usgs["time"], errors="coerce").dt.strftime("%Y-%m-%d")
usgs = usgs[(pd.to_datetime(usgs['time']) > pd.to_datetime('1960-01-01')) & (pd.to_datetime(usgs['time']) < pd.to_datetime('2023-01-01'))]

usgs['longitude'] = pd.to_numeric(usgs['longitude'], errors='coerce')
usgs['latitude'] = pd.to_numeric(usgs['latitude'], errors='coerce')
usgs['mag'] = pd.to_numeric(usgs['mag'], errors='coerce')

#filter the dataset by X > -123 and X < -113 and Y > 29 and Y < 39
usgs = usgs[usgs['longitude'] > -123]
usgs = usgs[usgs['longitude'] < -113]
usgs = usgs[usgs['latitude'] < 39]
usgs = usgs[usgs['latitude'] > 29]
usgs.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource\r
240,2022-12-31,33.397500,-116.393333,3.88,4.14,mw,132,16,0.07391,0.19,...,2023-09-22T21:50:30.029Z,"16 km N of Borrego Springs, CA",earthquake,0.1,0.38,NaN,6,reviewed,ci,ci\r
241,2022-12-31,34.355667,-116.921833,4.73,3.47,mw,121,25,0.07845,0.15,...,2023-03-07T19:00:01.040Z,"11km SSE of Lucerne Valley, CA",earthquake,0.09,0.41,NaN,4,reviewed,ci,ci\r
246,2022-12-22,37.620167,-122.025000,3.82,3.34,mw,141,16,NaN,0.16,...,2023-04-20T04:34:00.806Z,"3km N of Union City, CA",earthquake,0.1,0.17,NaN,3,reviewed,nc,nc\r
262,2022-12-17,37.918167,-122.304000,5.48,3.57,mw,170,19,0.01598,0.15,...,2023-07-27T08:15:34.318Z,"1km ENE of El Cerrito, CA",earthquake,0.1,0.17,NaN,4,reviewed,nc,nc\r
263,2022-12-13,36.604667,-121.209333,8.88,3.28,ml,67,55,0.03812,0.09,...,2023-02-18T22:04:08.040Z,"10km NW of Pinnacles, CA",earthquake,0.14,0.28,0.129,72,reviewed,nc,nc\r


In [81]:
time = []
for i in usgs['time']:
    time.append(pd.to_datetime(i))
usgs['time'] = time

## Data Grouping
Data is grouped into 1 week chunks

In [82]:
usgs_grouped_counts = pd.DataFrame(usgs.groupby(usgs['time'].dt.to_period('M')).mag.count())
usgs_grouped_counts.rename(columns={'mag':'count'}, inplace=True)

usgs_grouped_max = pd.DataFrame(usgs.groupby(usgs['time'].dt.to_period('M')).mag.max())

In [83]:
from sklearn.preprocessing import StandardScaler

## Data Scaling and Merging
Data is scaled using a Standard Scaler and merged

In [84]:
scaler = StandardScaler()

x_vals = np.array(usgs_grouped_counts["count"]).reshape(-1,1)
scaled_counts_arr = scaler.fit_transform(x_vals)

x_vals = np.array(usgs_grouped_max["mag"]).reshape(-1,1)
scaled_max_arr = scaler.fit_transform(x_vals)

In [85]:
merged_earthquake_data = usgs_grouped_counts.copy()
merged_earthquake_data['count'] = scaled_counts_arr
merged_earthquake_data['mag'] = scaled_max_arr
merged_earthquake_data = merged_earthquake_data.reset_index()
merged_earthquake_data.head()

,time,count,mag
0,1960-01,-0.371188,0.433925
1,1960-02,-0.430328,-1.353033
2,1960-03,-0.459898,-1.530252
3,1960-04,-0.415543,-0.688462
4,1960-05,-0.459898,-1.042900


In [86]:
import plotly.express as px
import plotly.graph_objects as go

In [87]:
merged_earthquake_data['time'] = merged_earthquake_data['time'].dt.to_timestamp()

fig = px.line(merged_earthquake_data, x='time', y=['count', 'mag'], labels={'value': 'Value'}, title='Scaled Earthquake Magnitude vs Multiplicity')
fig.update_layout(
    width=900,
    height=600,
)

fig.update_traces(line=dict(width=1.0))
fig.update_xaxes(rangeslider_visible=True)
fig.update_layout(dragmode='pan')

fig.show()

## Correlation Between Magnitude and Multiplicity

General correlation between magnitude and multiplicity

In [88]:
correlation_coefficient = merged_earthquake_data['count'].corr(merged_earthquake_data['mag'])
print(correlation_coefficient)

0.49935323787092756


Correlation between top 15 highest earthquake count months and the max earthquake magnitudes they had

In [89]:
scaled_earthquake_count = merged_earthquake_data.copy()
scaled_earthquake_count = merged_earthquake_data.nlargest(15, 'count')
correlation_coefficient = scaled_earthquake_count['count'].corr(scaled_earthquake_count['mag'])
print(correlation_coefficient)

0.3891397871624839


Correlation between top 15 largest earthquakes and the earthquake counts they had in that month

In [90]:
scaled_earthquake_mag = merged_earthquake_data.copy()
scaled_earthquake_mag = merged_earthquake_data.nlargest(15, 'mag')
correlation_coefficient = scaled_earthquake_mag['mag'].corr(scaled_earthquake_mag['count'])
print(correlation_coefficient)

0.660518445213043
